This script encompasses a workflow with randomly generated CT data. Used to demonstrate that Matlab and Python implementations lead to slightly different results when using spin_test(). 

In [24]:
import os
import enigmatoolbox
import numpy as np
import pandas as pd

from scipy.stats.mstats import spearmanr, pearsonr
from statsmodels.stats.multitest import fdrcorrection

In [25]:
# Import gene expression data, ordered as in original AHBA
os.chdir(r'C:\Users\Acer\Documents\Studium\PhD\01_MA_preterm_gene-expression\analysis\data\microarray')
genes = pd.read_csv('expression_brainorder.csv', index_col=0).T
reglabels = list(genes.index)
genelabels = list(genes.columns)
print("Your input gene list contains expression data for", len(genelabels), "across", len(reglabels), "regions...")
genes.head()

Your input gene list contains expression data for 15633 across 68 regions...


gene_symbol,A1BG,A1BG-AS1,A2M,A2ML1,A3GALT2,A4GALT,AAAS,AACS,AADACL3,AADAT,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
L_bankssts,0.440726,0.589278,0.381510,0.436797,0.467330,0.483065,0.452128,0.543946,0.536256,0.588966,...,0.486024,0.533246,0.522263,0.601487,0.587398,0.405509,0.634875,0.497207,0.666547,0.477138
L_caudalanteriorcingulate,0.678336,0.431307,0.572806,0.671694,0.480824,0.503408,0.355954,0.368647,0.556326,0.584441,...,0.387346,0.363805,0.343892,0.549072,0.387021,0.633337,0.349052,0.223801,0.406646,0.503784
L_caudalmiddlefrontal,0.520608,0.446375,0.609580,0.459314,0.501768,0.542337,0.516451,0.527325,0.416930,0.433428,...,0.486235,0.506752,0.408213,0.448193,0.453520,0.438250,0.429106,0.587044,0.404144,0.529791
L_cuneus,0.317040,0.307230,0.451889,0.477776,0.474542,0.471519,0.687771,0.399432,0.566165,0.440505,...,0.590275,0.526024,0.370078,0.550802,0.423469,0.451572,0.444684,0.829026,0.733465,0.387726
L_entorhinal,0.711983,0.261514,0.699565,0.514992,0.455938,0.476981,0.313874,0.296195,0.600913,0.489880,...,0.368767,0.372662,0.779778,0.435531,0.376815,0.562862,0.542200,0.085269,0.453389,0.591145


In [34]:
# random CT randomly generated in Matlab -> keep consistent
#raw_ct = np.random.uniform(low=2, high=3, size=(1,68))
#raw_ct = np.squeeze(raw_ct)
raw_ct = [2.58805912205776,2.47415092069538,2.91671074814889,2.70854757827065,2.73262913297386,2.01698900301724,2.11276871049553,2.66646281648323,2.41444090581014,2.02080723998185,2.04091515917065,2.01846176337805,2.54068235120194,2.98425707100468,2.07604808494799,2.88011775234679,2.83816330333713,2.32463650216453,2.74662763295216,2.25655974079322,2.90183144477258,2.22360150324691,2.98971228734974,2.05102334396157,2.76607876404913,2.70086328634064,2.20483752327492,2.49138462181976,2.64045953540602,2.10324598242545,2.63940741103014,2.50666531421732,2.64030120700556,2.20503425655978,2.57767854951135,2.21498410838450,2.80366321980188,2.96376106621246,2.04047398786189,2.70892046478552,2.98341210942531,2.37947296931904,2.14420901734182,2.62838728785446,2.83654282640552,2.96933843689782,2.22493682200663,2.49265154232225,2.57427236429738,2.43024895189780,2.57240342057358,2.96272977288615,2.25077821778144,2.70811834343147,2.21925017057708,2.02388433108541,2.23236613457015,2.83288178787764,2.87043287640347,2.33179605075937,2.21401196526758,2.36595076912635,2.41593958701836,2.89946720368525,2.93131599653450,2.02013578431943,2.38285745191099,2.27187183970819]

In [28]:
from enigmatoolbox.permutation_testing import spin_test, perm_sphere_p
numGenes = 20

# generate empty array with 3 columns for gene index, Spearman'r, pval, pval_fdr, pval_spin
corr_coeffs_CT = np.zeros([numGenes,5])

for g in range(0, numGenes):
#for g in range(0,len(genelabels)):
    print("Performing correlation for gene:", genelabels[g], "...")
    geneHere = genes.iloc[:,g]
    coef, p = spearmanr(geneHere, raw_ct)
    p_spin = spin_test(geneHere, raw_ct, surface_name='fsa5', parcellation_name='aparc', type='spearman', n_rot=1000, null_dist=False)
    
    #save values in array
    corr_coeffs_CT[g,0] = int(g)
    corr_coeffs_CT[g,1] = coef
    corr_coeffs_CT[g,2] = p
    corr_coeffs_CT[g,4] = p_spin


Performing correlation for gene: A1BG ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: A1BG-AS1 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: A2M ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutation 500 of 1000
permutation 600 of 1000
permutation 700 of 1000
permutation 800 of 1000
permutation 900 of 1000
permutation 1000 of 1000
Performing correlation for gene: A2ML1 ...
permutation 100 of 1000
permutation 200 of 1000
permutation 300 of 1000
permutation 400 of 1000
permutati

In [33]:
# view results
results = pd.DataFrame(corr_coeffs_CT)
results.columns = ['gene_symbol', 'Spearman_r', 'p', 'p_fdr', 'p_spin']
results['gene_symbol'] = genelabels[0:numGenes]

# adjust pval for FDR
pval_fdr = list(fdrcorrection(results.iloc[:,2].values)[1])
results['p_fdr'] = pval_fdr

# adjust pval_spin for FDR
pval_fdr = list(fdrcorrection(results.iloc[:,4].values)[1])
results['p_spin_fdr'] = pval_fdr
results.head(20)
# get significant genes based on p_spin
#results.sort_values(['p_spin'], ascending=False)

,gene_symbol,Spearman_r,p,p_fdr,p_spin,p_spin_fdr
0,A1BG,-0.133985,0.276020,0.709301,0.1715,0.381333
1,A1BG-AS1,-0.077108,0.531976,0.709301,0.2835,0.381333
2,A2M,-0.081154,0.510606,0.709301,0.2680,0.381333
3,A2ML1,-0.081536,0.508612,0.709301,0.2575,0.381333
4,A3GALT2,-0.080391,0.514604,0.709301,0.2860,0.381333
5,A4GALT,-0.369012,0.001957,0.039150,0.0005,0.010000
6,AAAS,0.022674,0.854379,0.899346,0.4365,0.436500
7,AACS,-0.041341,0.737832,0.857413,0.3385,0.391111
8,AADACL3,0.125740,0.306916,0.709301,0.1515,0.381333
9,AADAT,0.160171,0.191975,0.709301,0.0735,0.367500
